# 🧱 Build a Sales Data Mart (SQL Example)

This notebook demonstrates how to build a simple sales data mart using SQL logic in PySpark.

In [ ]:
# Install dependencies (if needed)
!pip install pyspark delta-spark

In [ ]:
# Initialize Spark with Delta support
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("Sales Data Mart") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## 🧾 Create Raw DataFrames (Simulating a Data Warehouse)

In [ ]:
orders = [(1, '2023-01-01', 101, 100.0, 5.0, 'store_1', 'emp_1'),
          (2, '2023-01-02', 102, 200.0, 10.0, 'store_2', 'emp_2')]
products = [(101, 'Shoes', 'Footwear'),
            (102, 'T-shirt', 'Apparel')]

df_orders = spark.createDataFrame(orders, ["order_id", "order_date", "product_id", "sales_amount", "discount", "store_id", "salesperson_id"])
df_products = spark.createDataFrame(products, ["product_id", "product_name", "category"])

df_orders.createOrReplaceTempView("raw_orders")
df_products.createOrReplaceTempView("raw_products")

## 🧱 Create the Sales Data Mart

In [ ]:
sales_mart = spark.sql("""
    SELECT
        o.order_id,
        o.order_date,
        p.product_name,
        p.category,
        o.sales_amount,
        o.discount,
        o.store_id,
        o.salesperson_id
    FROM raw_orders o
    JOIN raw_products p ON o.product_id = p.product_id
""")

sales_mart.show()

sales_mart.write.format("delta").mode("overwrite").save("file:///C:/data-lake/sales_mart")

## 📊 Query the Data Mart

In [ ]:
sales_mart.createOrReplaceTempView("sales_mart")
spark.sql("""
    SELECT category, SUM(sales_amount) AS total_revenue
    FROM sales_mart
    GROUP BY category
""").show()

In [ ]:
# Cleanup
spark.stop()